In [ ]:
import sys
import os
import numpy as np
from pathlib import Path
notebook_path = os.getcwd()  # Gets current working directory
project_root = os.path.abspath(os.path.join(notebook_path, '..'))
sys.path.insert(0, project_root)

data_dir = os.path.join(project_root, 'data')
raw_dir = os.path.join(data_dir, 'raw')
Path(raw_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from helper import get_dataset_with_copy

In [ ]:
jaguar_data_original, jaguar_data = get_dataset_with_copy(os.path.join(raw_dir, 'jaguar_movement_data.csv'))

jaguar_info_original, jaguar_info = get_dataset_with_copy(os.path.join(raw_dir, 'jaguar_additional_information.csv'))

In [ ]:
jaguar_data['timestamp'] = pd.to_datetime(jaguar_data['timestamp'], errors='coerce')
# Get unique jaguars
unique_jaguars = sorted(jaguar_data['individual.local.identifier (ID)'].unique())

# Create and display plot for each jaguar
for jaguar_id in unique_jaguars:
    # Create a copy of the data for this jaguar to avoid modifying original
    jaguar_subset = jaguar_data[jaguar_data['individual.local.identifier (ID)'] == jaguar_id].copy()
    
    # Create figure
    fig = plt.figure(figsize=(15, 10))  # Adjusted for better visibility
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    # Add map features
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue', alpha=0.5)
    ax.add_feature(cfeature.COASTLINE, linewidth=1.0)
    ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=1.0)
    
    # Add gridlines
    gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5)
    gl.xlabel_style = {'size': 12}
    gl.ylabel_style = {'size': 12}
    gl.top_labels = False
    gl.right_labels = False
    
    # Sort by timestamp if available to connect points in chronological order
    if 'timestamp' in jaguar_subset.columns:
        jaguar_subset = jaguar_subset.sort_values('timestamp')
    
    # Plot trajectory with both lines and points
    ax.plot(jaguar_subset['location.long'], 
            jaguar_subset['location.lat'],
            'b-',  # Blue line
            linewidth=1,
            alpha=0.7,
            transform=ccrs.PlateCarree(),
            zorder=1)
    
    # Add points
    ax.scatter(jaguar_subset['location.long'],
              jaguar_subset['location.lat'],
              c='red',  # Red points
              s=30,     # Point size
              alpha=0.6,
              transform=ccrs.PlateCarree(),
              zorder=2)
    
    # Mark start and end points
    ax.plot(jaguar_subset['location.long'].iloc[0],
            jaguar_subset['location.lat'].iloc[0],
            'go',  # Green circle for start
            markersize=10,
            label='Start',
            transform=ccrs.PlateCarree(),
            zorder=3)
    
    ax.plot(jaguar_subset['location.long'].iloc[-1],
            jaguar_subset['location.lat'].iloc[-1],
            'ro',  # Red circle for end
            markersize=10,
            label='End',
            transform=ccrs.PlateCarree(),
            zorder=3)
    
    # Set extent to focus on the actual movement area with some padding
    lon_min, lon_max = jaguar_subset['location.long'].min(), jaguar_subset['location.long'].max()
    lat_min, lat_max = jaguar_subset['location.lat'].min(), jaguar_subset['location.lat'].max()
    padding = 2  # degrees
    ax.set_extent([lon_min - padding, lon_max + padding, 
                   lat_min - padding, lat_max + padding], 
                   crs=ccrs.PlateCarree())
    
    # Add title
    plt.title(f'Jaguar {jaguar_id} Movement Trajectory', fontsize=16, pad=20)
    
    # Add information box
    info_text = f'Observations: {len(jaguar_subset)}'
    ax.text(0.02, 0.98, info_text,
            transform=ax.transAxes,
            fontsize=12,
            bbox=dict(facecolor='white', alpha=0.8),
            verticalalignment='top')
    
    # Add legend
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    plt.close()
    
    # Print movement statistics
    print(f"\nJaguar {jaguar_id} Movement Statistics:")
    print(f"Number of observations: {len(jaguar_subset)}")
    print(f"Movement area:")
    print(f"  Longitude: {lon_min:.2f}° to {lon_max:.2f}°")
    print(f"  Latitude: {lat_min:.2f}° to {lat_max:.2f}°")
    if 'timestamp' in jaguar_subset.columns:
        time_range = jaguar_subset['timestamp'].max() - jaguar_subset['timestamp'].min()
        print(f"Time span: {time_range}")
    print("-" * 50)